# import

In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.datasets import mnist
from time import time
import os
print(tf.__version__)

2.1.0


# Checkpoint function

In [2]:
def load(model, checkpoint_dir):
    print(" [*] Reading checkpoints...")

    ckpt = tf.train.get_checkpoint_state(checkpoint_dir)
    if ckpt :
        ckpt_name = os.path.basename(ckpt.model_checkpoint_path)
        checkpoint = tf.train.Checkpoint(dnn=model)
        checkpoint.restore(save_path=os.path.join(checkpoint_dir, ckpt_name))
        counter = int(ckpt_name.split('-')[1])
        print(" [*] Success to read {}".format(ckpt_name))
        return True, counter
    else:
        print(" [*] Failed to find a checkpoint")
        return False, 0

def check_folder(dir):
    if not os.path.exists(dir):
        os.makedirs(dir)
    return dir

# Data load & pre-processing function

In [3]:
def load_mnist() :
    (train_data, train_labels), (test_data, test_labels) = mnist.load_data()
    train_data = np.expand_dims(train_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]
    test_data = np.expand_dims(test_data, axis=-1) # [N, 28, 28] -> [N, 28, 28, 1]

    train_data, test_data = normalize(train_data, test_data)

    train_labels = to_categorical(train_labels, 10) # [N,] -> [N, 10]
    test_labels = to_categorical(test_labels, 10) # [N,] -> [N, 10]

    return train_data, train_labels, test_data, test_labels

def normalize(train_data, test_data):
    train_data = train_data.astype(np.float32) / 255.0
    test_data = test_data.astype(np.float32) / 255.0

    return train_data, test_data

# Performance function

In [4]:
def loss_fn(model, images, labels):
    logits = model(images, training=True)    
    loss = tf.reduce_mean(tf.keras.losses.categorical_crossentropy(y_pred=logits, y_true=labels, 
                                                                   from_logits=True))
    return loss

def accuracy_fn(model, images, labels):
    logits = model(images, training=False)
    prediction = tf.equal(tf.argmax(logits, -1), tf.argmax(labels, -1))
    accuracy = tf.reduce_mean(tf.cast(prediction, tf.float32))
    return accuracy

def grad(model, images, labels):
    with tf.GradientTape() as tape:
        loss = loss_fn(model, images, labels)
    return tape.gradient(loss, model.variables)

# Model function

In [5]:
def flatten() :
    return tf.keras.layers.Flatten()

def dense(label_dim, weight_init) :
    return tf.keras.layers.Dense(units=label_dim, use_bias=True, kernel_initializer=weight_init)

def sigmoid() :
    return tf.keras.layers.Activation(tf.keras.activations.sigmoid)

# Create model (class version)

In [6]:
class create_model_class(tf.keras.Model):
    def __init__(self, label_dim):
        super(create_model_class, self).__init__()
        weight_init = tf.keras.initializers.RandomNormal()

        self.model = tf.keras.Sequential()
        self.model.add(flatten())

        for i in range(2):
            self.model.add(dense(256, weight_init))
            self.model.add(sigmoid())

        self.model.add(dense(label_dim, weight_init))

    def call(self, x, training=None, mask=None):
        x = self.model(x)

        return x

# Create model (function version)

In [7]:
def create_model_function(label_dim) :
    weight_init = tf.keras.initializers.RandomNormal()

    model = tf.keras.Sequential()
    model.add(flatten())

    for i in range(2) :
        model.add(dense(256, weight_init))
        model.add(sigmoid())

    model.add(dense(label_dim, weight_init))

    return model

# Define data & hyper-parameter

In [8]:
""" dataset """
train_x, train_y, test_x, test_y = load_mnist()

""" parameters """
learning_rate = 0.001
batch_size = 128

training_epochs = 1
training_iterations = len(train_x) // batch_size

label_dim = 10

train_flag = True

""" Graph Input using Dataset API """
train_dataset = tf.data.Dataset.from_tensor_slices((train_x, train_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=batch_size).\
    batch(batch_size, drop_remainder=True)

test_dataset = tf.data.Dataset.from_tensor_slices((test_x, test_y)).\
    shuffle(buffer_size=100000).\
    prefetch(buffer_size=len(test_x)).\
    batch(len(test_x))

# Define model & optimizer & writer

In [9]:
""" Model """
network = create_model_function(label_dim)

""" Training """
optimizer = tf.keras.optimizers.Adam(learning_rate=learning_rate)

""" Writer """
checkpoint_dir = 'checkpoints'
logs_dir = 'logs'

model_dir = 'nn_softmax'

checkpoint_dir = os.path.join(checkpoint_dir, model_dir)
check_folder(checkpoint_dir)
checkpoint_prefix = os.path.join(checkpoint_dir, model_dir)
logs_dir = os.path.join(logs_dir, model_dir)

# Restore checkpoint & start train or test phase

In [10]:
if train_flag :

    checkpoint = tf.train.Checkpoint(dnn=network)

    # create writer for tensorboard
    summary_writer = tf.summary.create_file_writer(logdir=logs_dir)
    start_time = time()

    # restore check-point if it exits
    could_load, checkpoint_counter = load(network, checkpoint_dir)    

    if could_load:
        start_epoch = (int)(checkpoint_counter / training_iterations)        
        counter = checkpoint_counter        
        print(" [*] Load SUCCESS")
    else:
        start_epoch = 0
        start_iteration = 0
        counter = 0
        print(" [!] Load failed...")
    
    # train phase
    with summary_writer.as_default():  # for tensorboard
        for epoch in range(start_epoch, training_epochs):
            for idx, (train_input, train_label) in enumerate(train_dataset):            
                grads = grad(network, train_input, train_label)
                optimizer.apply_gradients(grads_and_vars=zip(grads, network.variables))

                train_loss = loss_fn(network, train_input, train_label)
                train_accuracy = accuracy_fn(network, train_input, train_label)
                
                for test_input, test_label in test_dataset:                
                    test_accuracy = accuracy_fn(network, test_input, test_label)

                tf.summary.scalar(name='train_loss', data=train_loss, step=counter)
                tf.summary.scalar(name='train_accuracy', data=train_accuracy, step=counter)
                tf.summary.scalar(name='test_accuracy', data=test_accuracy, step=counter)

                print(
                    "Epoch: [%2d] [%5d/%5d] time: %4.4f, train_loss: %.8f, train_accuracy: %.4f, test_Accuracy: %.4f" \
                    % (epoch, idx, training_iterations, time() - start_time, train_loss, train_accuracy,
                       test_accuracy))
                counter += 1                
        checkpoint.save(file_prefix=checkpoint_prefix + '-{}'.format(counter))
        
# test phase      
else :
    _, _ = load(network, checkpoint_dir)
    for test_input, test_label in test_dataset:    
        test_accuracy = accuracy_fn(network, test_input, test_label)

    print("test_Accuracy: %.4f" % (test_accuracy))

 [*] Reading checkpoints...
 [*] Failed to find a checkpoint
 [!] Load failed...
Epoch: [ 0] [    0/  468] time: 1.0642, train_loss: 2.34544945, train_accuracy: 0.0703, test_Accuracy: 0.0982
Epoch: [ 0] [    1/  468] time: 1.1509, train_loss: 2.28399849, train_accuracy: 0.0859, test_Accuracy: 0.1218
Epoch: [ 0] [    2/  468] time: 1.2447, train_loss: 2.28146839, train_accuracy: 0.2422, test_Accuracy: 0.2302
Epoch: [ 0] [    3/  468] time: 1.3414, train_loss: 2.29782844, train_accuracy: 0.1406, test_Accuracy: 0.1305
Epoch: [ 0] [    4/  468] time: 1.4312, train_loss: 2.25940037, train_accuracy: 0.1172, test_Accuracy: 0.1028
Epoch: [ 0] [    5/  468] time: 1.5189, train_loss: 2.30094957, train_accuracy: 0.0938, test_Accuracy: 0.1028
Epoch: [ 0] [    6/  468] time: 1.6137, train_loss: 2.27227259, train_accuracy: 0.2031, test_Accuracy: 0.1742
Epoch: [ 0] [    7/  468] time: 1.7064, train_loss: 2.28543854, train_accuracy: 0.2031, test_Accuracy: 0.2126
Epoch: [ 0] [    8/  468] time: 1.7952,

Epoch: [ 0] [   73/  468] time: 7.8500, train_loss: 1.02821946, train_accuracy: 0.7578, test_Accuracy: 0.7725
Epoch: [ 0] [   74/  468] time: 7.9438, train_loss: 1.02218175, train_accuracy: 0.7578, test_Accuracy: 0.7602
Epoch: [ 0] [   75/  468] time: 8.0345, train_loss: 1.03954792, train_accuracy: 0.6953, test_Accuracy: 0.7576
Epoch: [ 0] [   76/  468] time: 8.1322, train_loss: 1.02123523, train_accuracy: 0.7422, test_Accuracy: 0.7624
Epoch: [ 0] [   77/  468] time: 8.2230, train_loss: 1.04811573, train_accuracy: 0.7422, test_Accuracy: 0.7750
Epoch: [ 0] [   78/  468] time: 8.3148, train_loss: 0.97181165, train_accuracy: 0.7891, test_Accuracy: 0.7892
Epoch: [ 0] [   79/  468] time: 8.4035, train_loss: 0.83348900, train_accuracy: 0.8047, test_Accuracy: 0.7973
Epoch: [ 0] [   80/  468] time: 8.5053, train_loss: 0.98803985, train_accuracy: 0.7578, test_Accuracy: 0.8008
Epoch: [ 0] [   81/  468] time: 8.5960, train_loss: 0.84971625, train_accuracy: 0.8047, test_Accuracy: 0.8025
Epoch: [ 0

Epoch: [ 0] [  147/  468] time: 14.7904, train_loss: 0.53604329, train_accuracy: 0.8516, test_Accuracy: 0.8761
Epoch: [ 0] [  148/  468] time: 14.8832, train_loss: 0.56145346, train_accuracy: 0.8281, test_Accuracy: 0.8783
Epoch: [ 0] [  149/  468] time: 14.9739, train_loss: 0.49410817, train_accuracy: 0.8984, test_Accuracy: 0.8800
Epoch: [ 0] [  150/  468] time: 15.0697, train_loss: 0.55464423, train_accuracy: 0.8438, test_Accuracy: 0.8812
Epoch: [ 0] [  151/  468] time: 15.1595, train_loss: 0.58317864, train_accuracy: 0.8516, test_Accuracy: 0.8817
Epoch: [ 0] [  152/  468] time: 15.2592, train_loss: 0.50542760, train_accuracy: 0.8672, test_Accuracy: 0.8805
Epoch: [ 0] [  153/  468] time: 15.3539, train_loss: 0.50294554, train_accuracy: 0.8594, test_Accuracy: 0.8800
Epoch: [ 0] [  154/  468] time: 15.4477, train_loss: 0.49890548, train_accuracy: 0.8516, test_Accuracy: 0.8799
Epoch: [ 0] [  155/  468] time: 15.5384, train_loss: 0.43199307, train_accuracy: 0.8750, test_Accuracy: 0.8796
E

Epoch: [ 0] [  221/  468] time: 21.7020, train_loss: 0.32785499, train_accuracy: 0.8906, test_Accuracy: 0.9026
Epoch: [ 0] [  222/  468] time: 21.7997, train_loss: 0.34913176, train_accuracy: 0.9219, test_Accuracy: 0.9013
Epoch: [ 0] [  223/  468] time: 21.8934, train_loss: 0.45604229, train_accuracy: 0.8438, test_Accuracy: 0.8993
Epoch: [ 0] [  224/  468] time: 21.9862, train_loss: 0.27977383, train_accuracy: 0.9375, test_Accuracy: 0.8984
Epoch: [ 0] [  225/  468] time: 22.0760, train_loss: 0.42162642, train_accuracy: 0.8828, test_Accuracy: 0.8982
Epoch: [ 0] [  226/  468] time: 22.1697, train_loss: 0.42723185, train_accuracy: 0.8594, test_Accuracy: 0.8971
Epoch: [ 0] [  227/  468] time: 22.2635, train_loss: 0.28129926, train_accuracy: 0.9141, test_Accuracy: 0.8975
Epoch: [ 0] [  228/  468] time: 22.3622, train_loss: 0.44065398, train_accuracy: 0.8672, test_Accuracy: 0.8982
Epoch: [ 0] [  229/  468] time: 22.4559, train_loss: 0.45042524, train_accuracy: 0.8672, test_Accuracy: 0.8992
E

Epoch: [ 0] [  295/  468] time: 28.5486, train_loss: 0.33067319, train_accuracy: 0.8984, test_Accuracy: 0.9074
Epoch: [ 0] [  296/  468] time: 28.6544, train_loss: 0.31007040, train_accuracy: 0.9062, test_Accuracy: 0.9087
Epoch: [ 0] [  297/  468] time: 28.7481, train_loss: 0.30439764, train_accuracy: 0.9141, test_Accuracy: 0.9089
Epoch: [ 0] [  298/  468] time: 28.8448, train_loss: 0.31924111, train_accuracy: 0.9453, test_Accuracy: 0.9106
Epoch: [ 0] [  299/  468] time: 28.9356, train_loss: 0.39076224, train_accuracy: 0.8750, test_Accuracy: 0.9101
Epoch: [ 0] [  300/  468] time: 29.0313, train_loss: 0.27819562, train_accuracy: 0.9062, test_Accuracy: 0.9111
Epoch: [ 0] [  301/  468] time: 29.1211, train_loss: 0.33600774, train_accuracy: 0.8906, test_Accuracy: 0.9117
Epoch: [ 0] [  302/  468] time: 29.2149, train_loss: 0.26850432, train_accuracy: 0.9062, test_Accuracy: 0.9113
Epoch: [ 0] [  303/  468] time: 29.3046, train_loss: 0.24515624, train_accuracy: 0.9219, test_Accuracy: 0.9118
E

Epoch: [ 0] [  370/  468] time: 35.4661, train_loss: 0.33314866, train_accuracy: 0.8750, test_Accuracy: 0.9166
Epoch: [ 0] [  371/  468] time: 35.5649, train_loss: 0.23073165, train_accuracy: 0.9297, test_Accuracy: 0.9181
Epoch: [ 0] [  372/  468] time: 35.6546, train_loss: 0.21986178, train_accuracy: 0.9297, test_Accuracy: 0.9188
Epoch: [ 0] [  373/  468] time: 35.7494, train_loss: 0.28241742, train_accuracy: 0.9062, test_Accuracy: 0.9194
Epoch: [ 0] [  374/  468] time: 35.8381, train_loss: 0.24486753, train_accuracy: 0.9453, test_Accuracy: 0.9185
Epoch: [ 0] [  375/  468] time: 35.9299, train_loss: 0.21364045, train_accuracy: 0.9531, test_Accuracy: 0.9194
Epoch: [ 0] [  376/  468] time: 36.0197, train_loss: 0.33552545, train_accuracy: 0.9062, test_Accuracy: 0.9190
Epoch: [ 0] [  377/  468] time: 36.1134, train_loss: 0.20214280, train_accuracy: 0.9609, test_Accuracy: 0.9193
Epoch: [ 0] [  378/  468] time: 36.2111, train_loss: 0.31298101, train_accuracy: 0.9219, test_Accuracy: 0.9193
E

Epoch: [ 0] [  444/  468] time: 42.2430, train_loss: 0.26145995, train_accuracy: 0.9297, test_Accuracy: 0.9235
Epoch: [ 0] [  445/  468] time: 42.3328, train_loss: 0.27401400, train_accuracy: 0.8906, test_Accuracy: 0.9235
Epoch: [ 0] [  446/  468] time: 42.4215, train_loss: 0.28462294, train_accuracy: 0.9062, test_Accuracy: 0.9241
Epoch: [ 0] [  447/  468] time: 42.5143, train_loss: 0.27509457, train_accuracy: 0.9297, test_Accuracy: 0.9230
Epoch: [ 0] [  448/  468] time: 42.6040, train_loss: 0.14150533, train_accuracy: 0.9766, test_Accuracy: 0.9220
Epoch: [ 0] [  449/  468] time: 42.6918, train_loss: 0.29970765, train_accuracy: 0.8750, test_Accuracy: 0.9223
Epoch: [ 0] [  450/  468] time: 42.7856, train_loss: 0.31907177, train_accuracy: 0.9062, test_Accuracy: 0.9234
Epoch: [ 0] [  451/  468] time: 42.8733, train_loss: 0.19568716, train_accuracy: 0.9531, test_Accuracy: 0.9236
Epoch: [ 0] [  452/  468] time: 42.9651, train_loss: 0.29761481, train_accuracy: 0.9141, test_Accuracy: 0.9238
E

# Test accuracy : 92.41 %